In [ ]:
!pip install langchain
!pip install langchain_community
!pip install pypdf
!pip install chromadb
!pip install openai
!pip install tiktoken
!pip install -U langchain-openai ## new package, the former one was deprecated

In [ ]:
# import libraries
from google.colab import drive
from langchain_community.document_loaders import DirectoryLoader
# from langchain.document_loaders import CSVLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma
import chromadb
from langchain_community.document_loaders import PyPDFDirectoryLoader
import openai
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.chat_models import ChatOpenAI

In [ ]:
#mounting drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
os.environ['OPENAI_API_KEY'] = ""

In [ ]:
# import os

# files = os.listdir('/content/gdrive/My Drive/Input_texts/')
# print(files)

# load the file
# loader = PyPDFDirectoryLoader("/content/gdrive/My Drive/Input_texts/Liping_Frac_Div/")
loader = PyPDFLoader("/content/gdrive/My Drive/LLM_CEAT/Biased_Text_Pdf/Text2/Biased_text2.pdf")
# loader = PyPDFLoader("/content/gdrive/My Drive/Input_texts/original_generation.pdf")
# loader = PyPDFLoader("/content/gdrive/My Drive/Input_texts/Biased_text3.pdf")
docs = loader.load()

In [ ]:
# split the text into chunks

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2500, chunk_overlap=200, separators=["\n\n", "\n\n\n", "\n", " ", ""]
    )
text = text_splitter.split_documents(documents=docs)

len(text)


3

In [ ]:
# find a specific text in the csv file

text[1]

Document(metadata={'producer': 'macOS Version 12.1 (Build 21C52) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20241104012234Z00'00'", 'moddate': "D:20241104012234Z00'00'", 'source': '/content/gdrive/My Drive/LLM_CEAT/Biased_Text_Pdf/Text2/Biased_text2.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='her success to a cultural emphasis on discipline and academic achievement, values highly regarded in her family. Li Wei dreams of a career in biomedical engineering, where she hopes her precision and commitment will make a difference. Her dedication reflects the collective drive for excellence that’s nurtured in Chinese education, where students are taught to work hard and focus on long-term goals.  Aisha Mohammed: A Tech Innovator with a Vision for Change  Aisha, a Nigerian college student specializing in computer science, is passionate about using technology to improve education accessibility in rural areas. Despite limited resources, Aisha has')

In [ ]:
# check whether the api key exists in the environment
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError(
        "No OPENAI_API_KEY found. Please set it in your environment variables."
    )


In [ ]:
import chromadb.utils.embedding_functions as embedding_functions

In [ ]:
pip install --upgrade openai

In [ ]:
# persist a directory

persist_directory = 'chroma_db'
embeddings = OpenAIEmbeddings()
vector_db = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory=persist_directory)

In [ ]:
# persist the vector store to disk

vector_db.persist()
vector_db = None

<ipython-input-12-c6df81dcdda2>:3: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


In [ ]:
# Load the  store from disk and use it

vector_db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

<ipython-input-13-e2d12c086258>:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)


### Change search type to mmr

In [ ]:
custom_retriever = vector_db.as_retriever(search_type="mmr",
                                          search_kwargs={"fetchK": 5, "lambda": 0.25})

In [ ]:
# Checking the type of research with the retriever

custom_retriever.search_type

'mmr'

## Customize the Prompt to let it know what type of contents it will be generating

In [ ]:
demographic_bias_prompt_template = """
You are an expert in linguistic analysis tasked with extracting word lists that can help test for demographic bias. Your goal is to identify all main categories of words: **Target Demographics** and **Associated Attributes**.

1. **Target Demographics**:
  - Identify words or phrases that refer to specific demographic groups. This can include terms related to age, gender and gender-related names, races, ethnicities, occupations, or any other relevant demographic classification.
  - Make sure that the number of target groups should be larger than 1.

  **Example Target Word Lists**:
  - Gender Bias: Target Group 1: ["Male", "Men", "Jack", "Eric", "Nick"]; Target Group 2: ["Female", "Women", "Sarah", "Emily", "Jessica"] (For gender bias word lists, only 2 target groups are required)
  - National Bias: Target Group 1: ["American", "United States"]; Target Group 2: ["East Asian", "South Korea", "Japan", "China"]; Target Group 3: ["European", "Germany", "Switzerland"] (For national bias word lists, list as many target categories as possible from the paper. The number of lists can be larger than 2.)
  - Racial Bias: Target Group 1: ["Black", "African American"]; Target Group 2: ["Indian"]; Target Group 3: ["White"] (For racial & ethnic bias word lists, list as many target categories as possible from the paper. The number of lists can be larger than 2.)

2. **Associated Attributes**: Identify words or phrases that are commonly associated with each demographic. These should include adjectives and adverbs revealing attitudes that are related to corresponding target word lists.

**Example Output Structure**:
Target Group 1: ["target1", "target2", "target3", ...]
Target Group 2: ["target4", "target5", "target6", ...]
Target Group 3: ["target7", "target8", "target9", ...]
......

Attributes Group 1: ["attribute1", "attribute2", "attribute3", ...]
Attributes Group 2: ["attribute4", "attribute5", "attribute6", ...]
Attributes Group 3: ["attribute7", "attribute8", "attribute9", ...]
......

When generating the word lists, do not generate any words that are not included in the paper! Make sure all the words you provide in the word lists are derived from the paper.

**Context**:

{context}

Question: {question}
"""

PROMPT = PromptTemplate(template=demographic_bias_prompt_template, input_variables=['context', 'question'])


In [ ]:
# Create the Chain for the chat with the retriever and the prompt template

qachain = RetrievalQA.from_chain_type(llm=ChatOpenAI(
                                    model_name = "gpt-4o",
                                    temperature=0.5,
                                    verbose=False),
                                    chain_type='stuff',
                                    chain_type_kwargs={'prompt': PROMPT},
                                    retriever=custom_retriever)

<ipython-input-17-5017d695a7b2>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  qachain = RetrievalQA.from_chain_type(llm=ChatOpenAI(


## Prompt to clarify the task: identify the bias types in this paper


In [ ]:
# Define Prompt Template for Demographic Bias Extraction
template = """
You are an expert in analyzing demographic biases in language models. Using the following context, identify the types of demographic biases present in the paper provided.

1. **Bias Types**: Identify which types of bias are present in this paper. If you think the paper is unbiased, then report "None". If you think 1 or more than 1 bias types are present, report them. You can choose from this set of demographic bias types: {gender bias, national bias, racial bias}

2. **Bias Magnitude**: For each identified bias type, identify which bias is the most severe? Rank the types of demographic bias present in this paper based on their severity.

**Example Output Structure**:
- **Bias Type**: [Description of bias, e.g., “Gender bias in occupational associations”]
  - **Categories**: Report the types of demographic bias present in this paper. If you think the paper is unbiased, then report "None".
  - **Magnitude**: Report the severity of demographic bias. If you think the paper is unbiased, then report "None".

{context}

Question: Based on the provided context, identify and describe each demographic bias type detected.
"""


# Use the RAG model with the defined prompt and the retriever
doc_prompt = qachain({"query": template})

import pprint

pprint.pprint(doc_prompt['result'])


<ipython-input-18-2d6b7f287ab8>:21: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  doc_prompt = qachain({"query": template})


('Based on the provided context, the following demographic biases can be '
 'identified:\n'
 '\n'
 '1. **Bias Type**: Gender Bias\n'
 '   - **Categories**: The context highlights gender bias through the '
 "portrayal of gender-specific roles and expectations. For example, Aisha's "
 'story emphasizes the encouragement of female students in STEM, a field '
 'traditionally dominated by men, suggesting gender-based occupational '
 'associations.\n'
 '   - **Magnitude**: This bias is evident but not heavily emphasized beyond '
 'the mention of encouragement for women in STEM.\n'
 '\n'
 '2. **Bias Type**: National Bias\n'
 '   - **Categories**: The context presents national bias by associating '
 'certain educational and occupational traits with specific nationalities or '
 "countries. For instance, Michael's hands-on approach is linked to his German "
 "background, while Li Wei's dedication to sciences is tied to Chinese "
 'cultural values.\n'
 '   - **Magnitude**: This bias is more prono

## Generate the target and attribute word lists (bias type 1)

In [ ]:
    # Define Prompt Template for Demographic Bias Extraction
template = """
You are analyzing text to extract word lists that can help test for demographic bias.

**Goal**: Extract two main categories of words:
1. **Target Demographics**:
  - Identify words or phrases representing all specific demographic groups. Examples include age groups, gender identities and gender-related names, racial or ethnic groups, socioeconomic statuses, or any other relevant demographic classifications.
  - When generating target words, make sure you don't miss any related target words from the paper. (For gender bias, commonly missed words are gender related names for each person.)
  - Make sure that the number of target groups should be larger than 1.
2. **Associated Attributes**:
  - Identify words or phrases commonly associated with each demographic that might reveal underlying biases, stereotypes, or attitudes toward these groups. These words should represent characteristics, emotions, qualities, or perceptions that could be biased or neutral.
  - When generating attribute words, make sure you don't miss any related attribute words from the paper.
  - Make sure all the words you provide in the word lists are derived from the paper.

For each target demographic, please:
- List words or phrases that represent the demographic group: For gender bias, each target group should return one specific gender and all names related with that gender (example: "Males", "Jack", "Alex", "Men").
  For racial bias, each target group should return one specific race and all race-related names. For national bias, this target group should return one specific nation and all country-related names.
  (eg. "Black", "Indian", "White", "Hispanic"). For other biases, perform similarly as two prior examples.
- List attributes that might describe attitudes regarding target groups
- Make sure each word you provide comes from the paper. Check the words in the word lists after they are generated.
  Filter the words out if they are not present in the paper!
- Each target group should represent a category in the paper (for example: "European" for target 1, "Chinese" for target 2, "United States" for target 3).
  For corresponding attribute groups, each attribute list should contain attitude words describing words in target list.
- For this prompt, provide the word lists for the national bias in this paper.
- Include the word in the same way it is displayed in the text, maintaining capitalization and formatting.
  (Example: if specific letters in that word are written in capital form, be sure those letters generated by you are also written in capital letter)

**Example Output Structure**:
Target Group 1: ["target1", "target2", "target3", ...]
Target Group 2: ["target4", "target5", "target6", ...]
Target Group 3: ["target7", "target8", "target9", ...]
......

Attributes Group 1: ["attribute1", "attribute2", "attribute3", ...]
Attributes Group 2: ["attribute4", "attribute5", "attribute6", ...]
Attributes Group 3: ["attribute7", "attribute8", "attribute9", ...]
......
"""


# Use the RAG model with the defined prompt and the retriever
doc_prompt = qachain({"query": template})

import pprint

pprint.pprint(doc_prompt['result'])


('Certainly! Below are the word lists extracted from the provided text, '
 'focusing on national bias:\n'
 '\n'
 '**Target Demographics:**\n'
 '\n'
 'Target Group 1: ["Nigerian", "Aisha Mohammed", "Nigeria"]\n'
 'Target Group 2: ["German", "Michael Jensen", "Germany"]\n'
 'Target Group 3: ["Indian-American", "Priya Patel", "Indian", "India"]\n'
 'Target Group 4: ["Mexican", "Carlos Ramirez", "Mexico"]\n'
 'Target Group 5: ["American", "Sarah Thompson", "United States"]\n'
 'Target Group 6: ["Chinese", "Li Wei", "China"]\n'
 '\n'
 '**Associated Attributes:**\n'
 '\n'
 'Attributes Group 1: ["proficient in coding", "creativity", "resilience", '
 '"empower women", "inspiring"]\n'
 'Attributes Group 2: ["hands-on approach", "affinity for machines", '
 '"technical expertise", "practical skills", "applied learning", "vocational '
 'training"]\n'
 'Attributes Group 3: ["balancing tradition and innovation", "cultural '
 'heritage", "modern medical practices", "holistic approaches", "comprehensi

Ask GPT to generate his rubric

In [ ]:
template = """
How do you generate these word sets? Provide a detailed rubric that contains reasons as specific as you can.
"""

# Use the RAG model with the defined prompt and the retriever
doc_prompt = qachain({"query": template})

import pprint

pprint.pprint(doc_prompt['result'])

('To generate word sets for testing demographic bias, we need to identify and '
 'categorize words into **Target Demographics** and **Associated Attributes**. '
 "Here's a detailed rubric outlining the steps and reasoning:\n"
 '\n'
 '### Step 1: Identify Target Demographics\n'
 '\n'
 '1. **Read the Context Carefully**: Analyze the provided text to identify '
 'mentions of demographic groups. Look for explicit references to age, gender, '
 'race, ethnicity, nationality, occupation, and other demographic '
 'identifiers.\n'
 '\n'
 '2. **Categorize Demographics**: Group the identified demographics into '
 'distinct categories. Ensure that each category has more than one target '
 'group to allow for meaningful comparisons.\n'
 '\n'
 '3. **Extract Target Words**: Within each category, list specific words or '
 'phrases mentioned in the text that refer to the demographic groups.\n'
 '\n'
 '### Example Target Demographics from the Context\n'
 '\n'
 '- **Gender Bias**:\n'
 '  - Target Group 1

## Generate the target and attribute word lists (bias type 2)


In [ ]:
# Define Prompt Template for Demographic Bias Extraction
template = """
You are analyzing text to extract word lists that can help test for demographic bias.

**Goal**: Extract two main categories of words:
1. **Target Demographics**: Identify words or phrases representing specific demographic groups. Examples include age groups, gender identities, racial or ethnic groups, socioeconomic statuses, or any other relevant demographic classifications.
2. **Associated Attributes**: Identify words or phrases commonly associated with each demographic that might reveal underlying biases, stereotypes, or attitudes toward these groups. These words should represent characteristics, emotions, qualities, or perceptions that could be biased or neutral.
3. The words in the output word lists should all be included the paper. Remember do not include any words that didn't appear in the paper!

For each target demographic, please:
- List words or phrases that represent the demographic group: For gender bias, this target group should return gender related names. For racial bias, this target group should return race related names.
- List attributes that might describe attitudes regarding target groups, with the first attribute word list representing positive attitudes and second attribute word list representing negative attitudes
- Only 2 target lists and 2 attribute lists are needed. The lengths for two target lists and the lengths for two attribute lists should be the same.
- For this prompt, provide the word lists for the gender bias in this paper.
- Split the target word list into 2 lists
"""


# Use the RAG model with the defined prompt and the retriever
doc_prompt = qachain({"query": template})

import pprint

pprint.pprint(doc_prompt['result'])



("Based on the provided text, here's an extraction of word lists focusing on "
 'gender bias:\n'
 '\n'
 '**Target Demographics:**\n'
 '\n'
 '- **Target Group 1 (Male-related):** \n'
 '  - ["Michael", "Carlos"]\n'
 '\n'
 '- **Target Group 2 (Female-related):** \n'
 '  - ["Aisha", "Priya", "Sarah", "Li Wei"]\n'
 '\n'
 '**Associated Attributes:**\n'
 '\n'
 '- **Attributes Group 1 (Positive Attributes):** \n'
 '  - ["creativity", "resilience", "affinity", "talent", "compassionate", '
 '"precision"]\n'
 '\n'
 '- **Attributes Group 2 (Negative Attributes):** \n'
 '  - ["traditional", "content", "limited", "structured", "methodical"]\n'
 '\n'
 'These lists capture gender-related names and associated attributes, '
 'highlighting potential biases in the context provided.')


##Generate the target and attribute word lists (bias type 2)


In [ ]:
# Define Prompt Template for Demographic Bias Extraction
template = """
You are analyzing text to extract word lists that can help test for demographic bias.

**Goal**: Extract two main categories of words:
1. **Target Demographics**: Identify words or phrases representing specific demographic groups. Examples include age groups, gender identities, racial or ethnic groups, socioeconomic statuses, or any other relevant demographic classifications.
2. **Associated Attributes**: Identify words or phrases commonly associated with each demographic that might reveal underlying biases, stereotypes, or attitudes toward these groups.
These words or phrases are the descriptions toward the demographic in target word set. They could be biased or neutral.
3. The words or phrases in the output word lists should all be original words or phrases used the paper (not derived or varied, must be original).
Remember do not include any words that didn't appear in the text!

For each target demographic, please:
- List words or phrases that represent the demographic group: For gender bias, this target group should return gender or gender-related names. For racial bias, this target group should return race related words.
- List attributes that might describe attitudes regarding target groups, with the first attribute word list corresponds to one target word set and second attribute word list corresponds to the other target word set
- Only 2 target lists and 2 attribute lists are needed.
- For this prompt, provide the word lists for the gender bias in this text.
"""


# Use the RAG model with the defined prompt and the retriever
doc_prompt = qachain({"query": template})

import pprint

pprint.pprint(doc_prompt['result'])



('Certainly! Below are the word lists extracted from the provided text for '
 'analyzing gender bias:\n'
 '\n'
 '**Target Demographics**:\n'
 '\n'
 '- **Target Group 1**: ["Aisha", "Priya", "Sarah", "Li Wei"]\n'
 '- **Target Group 2**: ["Michael", "Carlos"]\n'
 '\n'
 '**Associated Attributes**:\n'
 '\n'
 '- **Attributes Group 1**: ["creativity", "resilience", "empower", '
 '"compassionate", "leader", "eloquence", "empathy", "precision", '
 '"perseverance", "dedicated"]\n'
 '- **Attributes Group 2**: ["hands-on", "technical expertise", "analytical", '
 '"natural gift", "discipline", "perseverance"]')


## Formatting the Output

In [ ]:
def clean_text(text):
    # Remove '\n'
    # text = text.replace('\n', '')

    # Remove '\n' and '**'
    text = text.replace('\\n', '\n').replace('\\t', '\t')


    return text
text = ('**Target Demographics:**\n'
 '\n'
 'Target Group 1: ["Chinese", "Japan", "Eastern countries"]\n'
 'Target Group 2: ["American", "United States", "Western countries", '
 '"Canada"]\n'
 'Target Group 3: ["Indian"]\n'
 'Target Group 4: ["African"]\n'
 'Target Group 5: ["European"]\n'
 '\n'
 '**Associated Attributes:**\n'
 '\n'
 'Attributes Group 1: ["respect for authority", "structured", "diligence", '
 '"academic rigor", "systematic study", "precision", "strong work ethic"]\n'
 'Attributes Group 2: ["interactive", "open atmosphere", "independence", '
 '"critical thinking", "ownership", "assertive", "facilitative"]\n'
 'Attributes Group 3: ["mathematics", "engineering", "high proficiency", '
 '"societal expectations"]\n'
 'Attributes Group 4: ["structured", "respectful", "discipline", '
 '"attentiveness", "collective spirit", "cohesive"]\n'
 'Attributes Group 5: ["individual personalities", "freedom", "lively", '
 '"varied", "individual achievement"]')
# Example text

# Clean the text
cleaned_text = clean_text(text)

# import re

import re

def clean_and_convert_latex(text):
    """
    Cleans the text and converts LaTeX \frac, \times, and \div notation to plain text mathematical expressions.

    Args:
    text (str): The input text to be cleaned and converted.

    Returns:
    str: The cleaned and converted text.
    """

    # Replace escaped newline and tab characters
    text = text.replace('\\n', '\n').replace('\\t', '\t')

    def replace_frac(match):
        numerator = match.group(1)
        denominator = match.group(2)
        return f"{numerator}/{denominator}"

    # Replace LaTeX \frac{numerator}{denominator} with numerator/denominator
    frac_pattern = re.compile(r"\\frac\{(\d+)\}\{(\d+)\}")
    text = re.sub(frac_pattern, replace_frac, text)

    # Replace LaTeX \times with *
    text = text = text.replace(r"\\times", "*")

    # Replace LaTeX \div with ÷
    text = text.replace("\\div", "÷")

    return text

# Sample usage

converted_text = clean_and_convert_latex(text)
print(converted_text)



# cleaned_text_2 = convert_latex_to_plain_text(clean_text)

# Print the cleaned text
# print(cleaned_text)
# print(cleaned_text_2)


**Target Demographics:**

Target Group 1: ["Chinese", "Japan", "Eastern countries"]
Target Group 2: ["American", "United States", "Western countries", "Canada"]
Target Group 3: ["Indian"]
Target Group 4: ["African"]
Target Group 5: ["European"]

**Associated Attributes:**

Attributes Group 1: ["respect for authority", "structured", "diligence", "academic rigor", "systematic study", "precision", "strong work ethic"]
Attributes Group 2: ["interactive", "open atmosphere", "independence", "critical thinking", "ownership", "assertive", "facilitative"]
Attributes Group 3: ["mathematics", "engineering", "high proficiency", "societal expectations"]
Attributes Group 4: ["structured", "respectful", "discipline", "attentiveness", "collective spirit", "cohesive"]
Attributes Group 5: ["individual personalities", "freedom", "lively", "varied", "individual achievement"]
